<a href="https://colab.research.google.com/github/nadjapereira/python-applications/blob/master/aula5_parte5_desafio4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 4

## 1. Introdução ao Pandas 

Já visto nas aulas anteriores.

## 2. Carregar o arquivo JSON


In [0]:
import simplejson as json
import pandas as pd

In [0]:
dados = []
with open('tweets_5min.json') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha))

In [0]:
dados[0]

In [0]:
df = pd.DataFrame(dados)

In [0]:
print(df.columns)

In [0]:
df.head()

In [0]:
print(len(df))

## 3. Definir o que iremos salvar!

Vamos salvar:

* text
* created_at
* coordinates
* retweet_count
* user -> screen_name
* user -> location
* user -> lang
* user -> followers_count



In [0]:
print("--> Dados do tweet")
print(df['text'][0])
print(df['created_at'][0])
print(df['coordinates'][0])
print(df['retweet_count'][0])

print("\n--> Dados do usuário")
print(df['user'][0]['screen_name'])
print(df['user'][0]['location'])
print(df['user'][0]['lang'])
print(df['user'][0]['followers_count'])


Agora vamos criar uma lista chamada colunas para salvar os nomes das colunas que queremos ter em nosso DataFrame.

In [0]:
colunas = ['text', 'created_at', 'coordinates', 'retweet_count', 'screen_name', 
           'location', 'lang', 'followers_count']

In [0]:
print(len(colunas))

Vamos criar um dataframe auxiliar para salvar somente os dados que queremos.

In [0]:
df_aux = pd.DataFrame(columns=colunas)
df_aux

Agora iremos adicionar apenas uma linha para entender como essa estrutura funciona.

Lembre-se que estamos recuperando informações do ```user```, sendo que os valores estão salvos em uma única coluna.

In [0]:
dados = [df['text'][0],
         df['created_at'][0],
         df['coordinates'][0],
         df['retweet_count'][0],
         df['user'][0]['screen_name'],
         df['user'][0]['location'],
         df['user'][0]['lang'],
         df['user'][0]['followers_count']
        ]

In [0]:
series_aux = pd.Series(dados, index=colunas)
df_aux = df_aux.append(series_aux, ignore_index=True)

É importante notar que se a cedula acima for executada mais de uma vez, o append irá adicionar repetidos. Tome cuidado!

In [0]:
df_aux

## 4. Entender o módulo Geopy e criar função que retorna a latitude e longitude



In [0]:
from geopy.geocoders import Nominatim

In [0]:
print(dir(Nominatim))

In [0]:
geolocalizador = Nominatim()
localizacao = geolocalizador.geocode("Av. Paulista, 302, São Paulo")
print(localizacao)

In [0]:
print(localizacao.address)

In [0]:
print(localizacao.latitude, localizacao.longitude)

In [0]:
print(localizacao.raw)

**Exercício - **Crie uma função que receba uma localização e retorne a latitude e longitude. Lembre-se de tratar possíves erros de conexão. a partir de um local, conseguimos recuperar a latitude e longitude.

In [0]:
from geopy.geocoders import Nominatim
def pegar_lat_long(local):
    geolocalizador = Nominatim()
    try:
        localizacao = geolocalizador.geocode(local)
        return (localizacao.latitude, localizacao.longitude)
    except:
        return 0
    

In [0]:
pegar_lat_long('Sao Paulo, SP')

## 5. Preparar os dados para a visualização

### Hashtags

Agora iremos criar uma função que irá salvar as hashtags utilizadas no texto.

In [0]:
palavras = df['text'][0]
palavras.split()

In [0]:
for palavra in palavras.split():
    if palavra.startswith('#'):
        print(palavra)

** Exercicio - Crie uma função que salvar as hashtags separadas por um espaço em branco. Exemplo:**

**Entrada:** ```Estou #programando em #python```

**Saida:** ```'#programando #python'```
    

In [0]:
' '.join(['1', '2'])

In [0]:
def salvar_hashtags2(texto):
    return ' '.join([x for x in texto.split() if x.startswith('#')])

In [0]:
salvar_hashtags2('RT @tonyojeda3: An Introduction to #MachineLearning with #Python https://t.co/TMrLLJNskx #DataScience #BigData')

In [0]:
def salvar_hashtags(texto):
    palavras = texto.split()
    aux = []
    for palavra in palavras:
        if palavra.startswith('#'):
            aux.append(palavra)
    converter = ' '.join(aux)
    return converter

In [0]:
salvar_hashtags(df['text'][0])

### Criar as novas colunas no DataFrame

Vamos criar as 3 novas colunas em nosso DataFrame.



In [0]:
colunas = ['text', 'created_at', 'coordinates', 'retweet_count', 
           'screen_name', 'location', 'lang', 'followers_count', 
           'lat', 'long', 'hastags']

In [0]:
df_aux = pd.DataFrame(columns=colunas)
len(df_aux.columns)

In [0]:
df_aux

Novamente vamos adicionar apenas um único item para verificar se está tudo correto.

In [0]:
latlong = pegar_lat_long(df['user'][3]['location'])
latlong

In [0]:
dados = [
    df['text'][3],
    df['created_at'][3],
    df['coordinates'][3],
    df['retweet_count'][3],
    df['user'][3]['screen_name'],
    df['user'][3]['location'],
    df['user'][3]['lang'],
    df['user'][3]['followers_count'],
    latlong[0],
    latlong[1],
    salvar_hashtags(df['text'][3])
    ]

In [0]:
dados

In [0]:
series_aux = pd.Series(dados, index=colunas)
df_aux = df_aux.append(series_aux, ignore_index=True)

In [0]:
df_aux

### Repetir esse processo para todos os tweets que foram salvos!

Até agora só trabalhamos com uma única entrada de dados, que foi o primeiro twitter salvo no arquivo.

Agora precisamos verificar quais informações são realmente importante e não podem ser vazias.

Primeiro precisamos percorrer todo o conjunto de dados, linha a linha. Como podemos fazer isso?

In [0]:
df_aux = pd.DataFrame(columns=colunas)
df_aux

In [0]:
print(len(df))

In [0]:
import time

In [0]:
t0 = time.time()
for i in range(0, len(df)):
    if df['user'][i]['location'] != None:
        tem_lat_long = pegar_lat_long(df['user'][i]['location'])
        if tem_lat_long != 0:
            dados = [
                df['text'][i],
                df['created_at'][i],
                df['coordinates'][i],
                df['retweet_count'][i],
                df['user'][i]['screen_name'],
                df['user'][i]['location'],
                df['user'][i]['lang'],
                df['user'][i]['followers_count'],
                tem_lat_long[0],
                tem_lat_long[1],
                salvar_hashtags(df['text'][i])
            ]
            print(i,end=" ")
            series = pd.Series(dados,index=colunas)
            df_aux = df_aux.append(series, ignore_index=True)
tf = time.time() - t0
print("\n\nTempo total para o parse foi de {} minutos".format(round(tf/60, 3)))

In [0]:
len(df_aux)

In [0]:
df_aux.head()

## 6. Salvar os dados em CSV

In [0]:
print(df_aux.count())

In [0]:
df_aux.to_csv('tweets_5min.csv', sep=';', encoding='utf-8', index=False)